In [1]:
# Import the dependencies.
import pandas as pd

# Import the requests library.
import requests

In [22]:
# Set seasons for which to pull data
season1 = 20192020
season2 = season1 - 10001
season3 = season2 - 10001

20172018

In [20]:
# first, retrieve all team IDs/API codes
# second, retrieve all rosters
# third, retrieve stats for each player for each of the 3 seasons - if "splits" empty, then return 0 for stats

In [13]:
# Create endpoint URL for team info
team_url = 'https://statsapi.web.nhl.com/api/v1/teams'

# Run API request for info on all teams
team_info = requests.get(team_url).json()

# Parse JSON to retrieve IDs for all active teams
team_data = []

for team in team_info["teams"]:
    team_data.append({"ID": str(team["id"]),
                      "Team": team["abbreviation"]})
team_data

[{'ID': '1', 'Team': 'NJD'},
 {'ID': '2', 'Team': 'NYI'},
 {'ID': '3', 'Team': 'NYR'},
 {'ID': '4', 'Team': 'PHI'},
 {'ID': '5', 'Team': 'PIT'},
 {'ID': '6', 'Team': 'BOS'},
 {'ID': '7', 'Team': 'BUF'},
 {'ID': '8', 'Team': 'MTL'},
 {'ID': '9', 'Team': 'OTT'},
 {'ID': '10', 'Team': 'TOR'},
 {'ID': '12', 'Team': 'CAR'},
 {'ID': '13', 'Team': 'FLA'},
 {'ID': '14', 'Team': 'TBL'},
 {'ID': '15', 'Team': 'WSH'},
 {'ID': '16', 'Team': 'CHI'},
 {'ID': '17', 'Team': 'DET'},
 {'ID': '18', 'Team': 'NSH'},
 {'ID': '19', 'Team': 'STL'},
 {'ID': '20', 'Team': 'CGY'},
 {'ID': '21', 'Team': 'COL'},
 {'ID': '22', 'Team': 'EDM'},
 {'ID': '23', 'Team': 'VAN'},
 {'ID': '24', 'Team': 'ANA'},
 {'ID': '25', 'Team': 'DAL'},
 {'ID': '26', 'Team': 'LAK'},
 {'ID': '28', 'Team': 'SJS'},
 {'ID': '29', 'Team': 'CBJ'},
 {'ID': '30', 'Team': 'MIN'},
 {'ID': '52', 'Team': 'WPG'},
 {'ID': '53', 'Team': 'ARI'},
 {'ID': '54', 'Team': 'VGK'}]

In [18]:
# Creating list for player data
player_data = []

# Run API call for each team to get their roster information, then parse roster JSON to retrieve player data
for team in team_data:
    
    # Creating URL for team roster
    roster_url = team_url + "/" + team['ID'] + "/roster"
    
    # API call for team roster in JSON format
    roster_info = requests.get(roster_url).json()
    
    # Retrieving data for each player on roster
    for player in roster_info['roster']:
        player_data.append({'player_id': str(player['person']['id']),
                            'Player Name': player['person']['fullName'], 
                            'Team': team['Team'],
                            'Position': player['position']['abbreviation']})
player_data

[{'player_id': '8475222',
  'Player Name': 'Sami Vatanen',
  'Team': 'NJD',
  'Position': 'D'},
 {'player_id': '8478406',
  'Player Name': 'Mackenzie Blackwood',
  'Team': 'NJD',
  'Position': 'G'},
 {'player_id': '8478447',
  'Player Name': 'Nicholas Merkley',
  'Team': 'NJD',
  'Position': 'RW'},
 {'player_id': '8480003',
  'Player Name': 'Jesper Boqvist',
  'Team': 'NJD',
  'Position': 'C'},
 {'player_id': '8480860',
  'Player Name': 'Kevin Bahl',
  'Team': 'NJD',
  'Position': 'D'},
 {'player_id': '8474056',
  'Player Name': 'P.K. Subban',
  'Team': 'NJD',
  'Position': 'D'},
 {'player_id': '8475151',
  'Player Name': 'Kyle Palmieri',
  'Team': 'NJD',
  'Position': 'RW'},
 {'player_id': '8475179',
  'Player Name': 'Dmitry Kulikov',
  'Team': 'NJD',
  'Position': 'D'},
 {'player_id': '8475809',
  'Player Name': 'Scott Wedgewood',
  'Team': 'NJD',
  'Position': 'G'},
 {'player_id': '8476807',
  'Player Name': 'Matt Tennyson',
  'Team': 'NJD',
  'Position': 'D'},
 {'player_id': '84768

In [19]:
# Separate goalies from skaters
goalie_data = []
skater_data = []

for player in player_data:
    if player['Position'] == 'G':
        goalie_data.append(player)
    else:
        skater_data.append(player)
goalie_data

[{'player_id': '8478406',
  'Player Name': 'Mackenzie Blackwood',
  'Team': 'NJD',
  'Position': 'G'},
 {'player_id': '8475809',
  'Player Name': 'Scott Wedgewood',
  'Team': 'NJD',
  'Position': 'G'},
 {'player_id': '8477480',
  'Player Name': 'Eric Comrie',
  'Team': 'NJD',
  'Position': 'G'},
 {'player_id': '8471239',
  'Player Name': 'Cory Schneider',
  'Team': 'NYI',
  'Position': 'G'},
 {'player_id': '8473575',
  'Player Name': 'Semyon Varlamov',
  'Team': 'NYI',
  'Position': 'G'},
 {'player_id': '8478009',
  'Player Name': 'Ilya Sorokin',
  'Team': 'NYI',
  'Position': 'G'},
 {'player_id': '8478048',
  'Player Name': 'Igor Shesterkin',
  'Team': 'NYR',
  'Position': 'G'},
 {'player_id': '8480382',
  'Player Name': 'Alexandar Georgiev',
  'Team': 'NYR',
  'Position': 'G'},
 {'player_id': '8470880',
  'Player Name': 'Brian Elliott',
  'Team': 'PHI',
  'Position': 'G'},
 {'player_id': '8479394',
  'Player Name': 'Carter Hart',
  'Team': 'PHI',
  'Position': 'G'},
 {'player_id': '8